In [1]:
!nvidia-smi

Wed Mar 23 23:07:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P4000        Off  | 00000000:8B:00.0 Off |                  N/A |
| 46%   29C    P8     8W / 105W |      2MiB /  8117MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Inštalácia knižníc

In [2]:
!pip install wandb

!pip install opencv-python
!pip install keras
!pip install scikit-plot

### Import knižníc

In [5]:
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
import wandb

from wandb.keras import WandbCallback
from sys import stdout
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score
from sklearn import metrics

### Prihlásenie sa do wandbu

In [6]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ishadowstormi (use `wandb login --relogin` to force relogin)


True

### Načítanie snímok

In [10]:
with open('train.pkl','rb') as f: X_train, train_df = pkl.load(f)
with open('test.pkl','rb') as f: X_test, test_df = pkl.load(f)
with open('valid.pkl','rb') as f: X_valid, valid_df = pkl.load(f)

y_train = train_df["category"]
y_test = test_df["category"]
y_valid = valid_df["category"]

In [11]:
print("TrainSetInto = {}".format(X_train.shape))
print("TestSetInto = {}".format(X_test.shape))
print("ValidSetInto = {}".format(X_valid.shape))

TrainSetInto = (4668, 299, 299, 1)
TestSetInto = (1191, 299, 299, 1)
ValidSetInto = (100, 299, 299, 1)


### Architektúra siete

In [13]:
def Model(conv_layer1, conv_layer2, conv_layer3, conv_layer4, dense):
    model = Sequential()
    
    model.add(Conv2D(conv_layer1, (3, 3), input_shape=[299, 299,1], activation='relu'))
    model.add(Conv2D(conv_layer1, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(conv_layer2, (3, 3), activation='relu'))
    model.add(Conv2D(conv_layer2, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(conv_layer3, (3, 3), activation='relu'))
    model.add(Conv2D(conv_layer3, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(conv_layer4, (3, 3), activation='relu'))
    model.add(Conv2D(conv_layer4, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(dense, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer="Adam",
                  loss="binary_crossentropy",
                  metrics=['accuracy'])
    
    return model

### Konfigurácia procesu optimalizácie 

In [17]:
def train():
    #defaul konfigurácia
    config_default={
        'conv_layer1': 32,
        'conv_layer2': 64,
        'conv_layer3': 64,
        'conv_layer4': 128,
        'dense': 512,
        'batch_size': 32,
        'architecture': 'CNN'
    }

    # inicializácia wandb projektu
    wandb.init(config=config_default)
    config = wandb.config

    # inicializácia vstupujúcich parametrov modelu
    model = Model(config.conv_layer1, config.conv_layer2, config.conv_layer3, config.conv_layer4, config.dense)
    
    # vytvorenie imagegeneratora (augmantácia trénovacej množiny)
    train_datagen = ImageDataGenerator(vertical_flip=True,horizontal_flip=True, brightness_range=[0.3,0.7])
    
#     _ = model.fit(X_train,
#                   y_train,
#                   epochs=config.epochs,
#                   batch_size=config.batch_size,
#                   validation_data = (X_test, y_test),
#                   callbacks = [WandbCallback(monitor='val_loss',
#                                              verbose = 1,
#                                              mode = 'min'
#                                             )])
#     STEPS_PER_EPOCH = y_train.size / config.batch_size
#     SAVE_PERIOD = 2

#     filepath = "../FrantisekUrban/wandb_callback_weights/saved-model-{epoch:02d}.h5"
#     checkpoint1 = ModelCheckpoint(filepath, verbose=1, save_best_only=False, save_weights_only=True, save_freq=int(SAVE_PERIOD * STEPS_PER_EPOCH))
    
    
    # konfigurácia procesu učenia
    _ = model.fit(train_datagen.flow(X_train,y_train,
                                       config.batch_size),
                  validation_data = (X_valid, y_valid),
                  epochs=28,  
                  callbacks = [WandbCallback(monitor='val_loss',
                                             verbose = 0,
                                             mode = 'min',
                                             save_model=False
                                            )])
    
    # vyhodnotenie vytvoreného modelu    
    print("\n-----Model evaluation on test data-----")
    loss, accuracy = model.evaluate(X_test, y_test)
    
    # vypočet a výpis chyby modelu
    print('Test Error Rate: ', round((1-accuracy)*100, 2))
    # odoslanie hodnoty chyby do wandbu
    wandb.log({'Test Error Rate': round((1-accuracy)*100, 2)})
    
    # predikcia modelu
    y_pred = model.predict(X_test)
    # výpočet hodnoty AUC
    roc = roc_auc_score(y_test, y_pred)
    
    # výpis a následne odoslanie AUC do wandbu
    print('AUC: ', round(roc,4))
    wandb.log({'AUC': round(roc,4)})
    
    # ukončenie daného sweepu
    wandb.finish()

In [18]:
# inicializácia jednotlivých typov parametrov modelu
sweep_config = {
  'method': 'grid', 
  'metric': {
      'name': 'val_loss',
      'goal': 'minimize'
  },
  'early_terminate':{
      'type': 'hyperband',
      'min_iter': 7
  },
  'parameters': {
      'conv_layer1': {
          'values': [16, 32]
      },
      'conv_layer2': {
          'values': [32, 64]
      },
      'conv_layer3': {
          'values': [64, 128]
      },
      'conv_layer4': {
          'values': [64, 128]
      },
      'dense':{
          'values': [512, 1024]
      },
      'batch_size': {
          'values': [32, 64]
      }
  }
}

In [19]:
# vytvorenie sweepu
sweep_id = wandb.sweep(sweep_config, project="Coronary angiography network")

Create sweep with ID: wco4rtlh
Sweep URL: https://wandb.ai/ishadowstormi/Coronary%20angiography%20network/sweeps/wco4rtlh


In [20]:
# spustenie procesu vyhľadávania optimálneho riešenia
wandb.agent(sweep_id, function=train)

wandb: Agent Starting Run: h6c8ios1 with config:
wandb: 	batch_size: 32
wandb: 	conv_layer1: 16
wandb: 	conv_layer2: 32
wandb: 	conv_layer3: 64
wandb: 	conv_layer4: 64
wandb: 	dense: 512
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/28
146/146 [==============================] - 20s 139ms/step - loss: 1.8789 - accuracy: 0.6512 - val_loss: 0.7938 - val_accuracy: 0.6100
Epoch 2/28
146/146 [==============================] - 19s 131ms/step - loss: 0.5445 - accuracy: 0.7277 - val_loss: 0.8052 - val_accuracy: 0.6100
Epoch 3/28
146/146 [==============================] - 19s 131ms/step - loss: 0.4961 - accuracy: 0.7569 - val_loss: 0.7560 - val_accuracy: 0.5500
Epoch 4/28
146/146 [==============================] - 19s 132ms/step - loss: 0.4533 - accuracy: 0.7826 - val_loss: 0.7041 - val_accuracy: 0.6800
Epoch 5/28
146/146 [==============================] - 19s 132ms/step - loss: 0.4525 - accuracy: 0.7802 - val_loss: 1.0005 - val_accuracy: 0.5300
Epoch 6/28
146/146 [==============================] - 19s 132ms/step - loss: 0.4220 - accuracy: 0.7999 - val_loss: 0.7140 - val_accuracy: 0.5700
Epoch 7/28
146/146 [==============================] - 19s 132ms/step - loss: 0.4145 - accuracy: 0.8031 - val_loss: 3.1913 - val_ac

AUC,▁
Test Error Rate,▁
accuracy,▁▃▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇█████
epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇██
loss,█▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▄▄▂▅▂▃▁▃▅▆▅▄▆▄▅█▇█▇▅█▄▄▅██▅█
val_loss,▂▂▂▁▂▂█▃▃▁▂▂▂▂▂▁▂▁▁▃▁▃▂▁▂▂▂▂
AUC,0.8801
Test Error Rate,24.18
accuracy,0.92759
best_epoch,17


wandb: Agent Starting Run: dvmo9xtz with config:
wandb: 	batch_size: 32
wandb: 	conv_layer1: 16
wandb: 	conv_layer2: 32
wandb: 	conv_layer3: 64
wandb: 	conv_layer4: 64
wandb: 	dense: 1024
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/28
146/146 [==============================] - 20s 137ms/step - loss: 1.7168 - accuracy: 0.6635 - val_loss: 6.5204 - val_accuracy: 0.5000
Epoch 2/28
146/146 [==============================] - 20s 134ms/step - loss: 0.4959 - accuracy: 0.7483 - val_loss: 1.1718 - val_accuracy: 0.5300
Epoch 3/28
146/146 [==============================] - 20s 134ms/step - loss: 0.4771 - accuracy: 0.7697 - val_loss: 1.5623 - val_accuracy: 0.5100
Epoch 4/28
146/146 [==============================] - 20s 134ms/step - loss: 0.4539 - accuracy: 0.7734 - val_loss: 0.6349 - val_accuracy: 0.6700
Epoch 5/28
146/146 [==============================] - 20s 134ms/step - loss: 0.4241 - accuracy: 0.7926 - val_loss: 1.7686 - val_accuracy: 0.6400
Epoch 6/28
146/146 [==============================] - 20s 134ms/step - loss: 0.4062 - accuracy: 0.7924 - val_loss: 1.9070 - val_accuracy: 0.6800
Epoch 7/28
146/146 [==============================] - 20s 135ms/step - loss: 0.3965 - accuracy: 0.8162 - val_loss: 1.2884 - val_ac

...

Run oicflfsc errored: ResourceExhaustedError()
wandb: ERROR Run oicflfsc errored: ResourceExhaustedError()
wandb: Agent Starting Run: e374p0zc with config:
wandb: 	batch_size: 64
wandb: 	conv_layer1: 32
wandb: 	conv_layer2: 64
wandb: 	conv_layer3: 128
wandb: 	conv_layer4: 128
wandb: 	dense: 512
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/28



Run e374p0zc errored: ResourceExhaustedError()
wandb: ERROR Run e374p0zc errored: ResourceExhaustedError()
wandb: Agent Starting Run: rprsjfdg with config:
wandb: 	batch_size: 64
wandb: 	conv_layer1: 32
wandb: 	conv_layer2: 64
wandb: 	conv_layer3: 128
wandb: 	conv_layer4: 128
wandb: 	dense: 1024
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/28



Run rprsjfdg errored: ResourceExhaustedError()
wandb: ERROR Run rprsjfdg errored: ResourceExhaustedError()
wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
